In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import datetime
from IPython.display import clear_output
from datetime import timedelta
from scipy.stats import skewnorm
import inspect, re
import h2o
from h2o.automl import H2OAutoML
import xgboost as xgb
import random
import gc
from sklearn.model_selection import train_test_split
from sklearn import  metrics
import time
import progressbar
import warnings
import math
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV as LogReg

               
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from WOE_Houston import *

In [2]:
fy_10 = pd.read_csv('fy_10_augmented.csv')
fy_11 = pd.read_csv('fy_11_augmented.csv')
fy_12 = pd.read_csv('fy_12_augmented.csv')
fy_13 = pd.read_csv('fy_13_augmented.csv')
fy_14 = pd.read_csv('fy_14_augmented.csv')
fy_15 = pd.read_csv('fy_15_augmented.csv')
fy_16 = pd.read_csv('fy_16_augmented.csv')

fy_list = [fy_10, fy_11, fy_12, fy_13, fy_14, fy_15, fy_16]

del fy_10
del fy_11
del fy_12
del fy_13
del fy_14
del fy_15
del fy_16
gc.collect()

essentials = ['Unnamed: 0',
 'Fund Id',
 'Business Area',
 'Fund Center Id',
 'GL Account',
 'GL Description',
 'GL Category',
 'GL Category Description',
 'Revenue or Expenditure',
 'Fiscal Year',
 'Fund Name',
 'Fund Type Descr',
 'Business Area Name',
 'Fund Center Name',
 'Commitment Set Description',
 'Original Budget',
 'Current Budget',
 'Actuals',
 '3Y_Past_Actuals',
 '2Y_Past_Actuals',
 '1Y_Past_Actuals',
 '2Y_1Y_Actuals_Mean',
 '3Y_2Y_1Y_Actuals_Mean',
 '3Y_Past_Original_Budget',
 '2Y_Past_Original_Budget',
 '1Y_Past_Original_Budget',
 '2Y_1Y_Original_Budget_Mean',
 '3Y_2Y_1Y_Original_Budget_Mean',
 '3Y_Past_OA_Difference',
 '2Y_Past_OA_Difference',
 '1Y_Past_OA_Difference',
 '2Y_1Y_OA_Difference_Mean',
 '3Y_2Y_1Y_OA_Difference_Mean',
 'OA_Difference',
 'Yearly Crime Index',
 'Yearly Avg High Temp',
 'Yearly Avg Low Temp',
 'Yearly Rain Fall',
 'Astros Playoff Rounds',
 'Astros Yearly Attendance',
 'Rockets Playoff Rounds',
 'Texans Playoff Rounds',
 'Texans Average Attendance',
 'Rockets Total Attendance']

dummies = ['Fund Name_Asset Forfeiture Fund - Justice',
 'Fund Name_Asset Forfeiture Fund - State',
 'Fund Name_Auto Dealers Fund',
 'Fund Name_Building Inspection Fund',
 'Fund Name_C&E-Civic Center Facility Revenue Fund',
 'Fund Name_Cable Television Fund',
 'Fund Name_Child Safety Fund',
 'Fund Name_Digital Houston Special Revenue',
 'Fund Name_Expansion of Recycling Program',
 'Fund Name_Fleet Management Fund',
 'Fund Name_GSD - In-House Renovation Revolving Fund',
 'Fund Name_General Fund',
 'Fund Name_Greater Houston Trans & Emer Center',
 'Fund Name_HAS-Revenue Fund',
 'Fund Name_Health Benefits Fund',
 'Fund Name_Historic Preservation Fund',
 'Fund Name_Houston Emergeny Center',
 'Fund Name_Long-Term Disablility Plan Fund',
 'Fund Name_Municipal Court Technology Fund',
 'Fund Name_PWE-Combined Utility System Gen Pur Fund',
 'Fund Name_PWE-Combined Utility System Oper Fund',
 'Fund Name_PWE-W & S System Operating Fund',
 'Fund Name_Park Special Revenue Fund',
 'Fund Name_Parking Mangement Operating Fund',
 'Fund Name_Police Special Services',
 'Fund Name_Project Cost Recovery',
 'Fund Name_Property & Casualty Fund',
 'Fund Name_Stormwater Fund',
 'Fund Name_Supplemental Environmental Proj',
 'Fund Name_Workers Compensation Admin Fund',
 'Fund Type Descr_Enterprise Funds',
 'Fund Type Descr_General Funds',
 'Fund Type Descr_Internal Services Funds',
 'Fund Type Descr_Special Revenue Funds',
 'Business Area Name_Admn. & Regulatory Affairs',
 'Business Area Name_Business Opportunity',
 "Business Area Name_City Controller's Office",
 'Business Area Name_City Council',
 'Business Area Name_City Secretary',
 'Business Area Name_Citywide General Government',
 'Business Area Name_Convention and Entertainment',
 'Business Area Name_Finance',
 'Business Area Name_General Debt Services',
 'Business Area Name_General Services Department',
 'Business Area Name_Health & Human Services',
 'Business Area Name_Housing and Community Devp.',
 'Business Area Name_Houston Airport System (HAS)',
 'Business Area Name_Houston Emergency Center (HEC)',
 'Business Area Name_Houston Fire Department (HFD)',
 'Business Area Name_Houston Police Department',
 'Business Area Name_Human Resources Dept.',
 'Business Area Name_Information Technology (IT)',
 'Business Area Name_Legal Department',
 'Business Area Name_Library',
 "Business Area Name_Mayor's Office",
 'Business Area Name_Municipal Court Judicial',
 'Business Area Name_Municipal Courts Department',
 'Business Area Name_Parks & Recreation',
 'Business Area Name_Planning & Development',
 'Business Area Name_Public Works & Engineering-PWE',
 'Business Area Name_Solid Waste Management',
 'Commitment Set Description_50 - Total Expenditures',
 'Business Area Name_Houston Police Department-HPD',
 'Business Area Name_Houston Information Tech Svcs'
]

drop_list = [
    
    'Fund Name', 
    'Fund Type Descr', 
    'Business Area Name',
    'Commitment Set Description',
    'GL Description',
    'GL Category Description',
    'Revenue or Expenditure',
    'Fund Type Descr',
    'Fund Center Name'
]


for i, y in enumerate(fy_list):
    fy_list[i] = pd.concat([y[essentials], y[dummies]], axis=1)
    fy_list[i]['Over Budget'] = fy_list[i]['OA_Difference']>0
    
for i, y in enumerate(fy_list):
    if i==0:
        fy_list[i]["1YP_OA_Diff_Over_80K"]=999999999999999
    else:
        fy_list[i]["1YP_OA_Diff_Over_80K"]=0
        over80k_index = fy_list[i].loc[(fy_list[i]['1Y_Past_OA_Difference'].apply(lambda x: abs(x))>80000.0)].index
        fy_list[i].loc[over80k_index, '1YP_OA_Diff_Over_80K'] = 1
        

        
        
splits_to_try = [
    
                #predicting off "two years"
                 [(0,1),(1,2),'10-11 to predict 11'],
                 [(0,2),(2,3),'10-11 to predict 12'],
                 [(0,3),(3,4),'10-12 to predict 13'],
                 #[(0,3),(3,5),'10-12 to predict 13 and 14'],
                 [(1,4),(4,5),'11-13 to predict 14'], 
                 #[(1,4),(4,6),'11-13 to predict 14 and 15'],
                 [(2,5),(5,6),'12-14 to predict 15'],
                 #[(2,5),(5,7),'12-14 to predict 15 and 16'],
                 [(3,6),(6,7),'13-15 to predict 16'], 
    
                #predicting off "three" years
    
                 [(0,4),(5,6),'10-13 to predict 14'],
                 #[(0,4),(4,6),'10-13 to predict 14 and 15'],
                 #[(0,4),(4,7),'10-13 to predict 14 and 15 and 16'],
                 [(1,5),(5,6),'11-14 to predict 15'],
                 #[(1,5),(5,7),'11-14 to predict 15 and 16'],
                 [(2,6),(6,7),'12-15 to predict 16']
                ]
        


In [3]:
!ls

01_Houston_Budget.ipynb
02_EDA.ipynb
03_Getting_Cleaned_CSVs.ipynb
04_H2o_Cleaned.ipynb
05_Adding_One_Hot.ipynb
06_Taking_Out_Fy16.ipynb
07_Adding_One_Hot_2.ipynb
08_Houston_EDA.ipynb
09_Adding_Past_Actuals.ipynb
10_xgb_h20_augmented.ipynb
11_adding_fy16.ipynb
12_xgb_h20_added_fy16.ipynb
13_xgb_split_80K.ipynb
14_xgb_other_splits.ipynb
15_xgb_classifier.ipynb
16_xgb_classifier_pcaWOE.ipynb
17_xgb_add_council_data.ipynb
18_classifier_regressor.ipynb
19_EDA_Pandas_Profiling_Report.ipynb
2010_clfpred
2010_council.csv
2010council
2010council.csv
2011_clfpred
2011_council.csv
2011council
2011council.csv
2012_clfpred
2012_council.csv
2012council
2012council.csv
2013_clfpred
2013_council.csv
2013council
2013council.csv
2014_clfpred
2014_council.csv
2014council
2014council.csv
2015_clfpred
2015_council.csv
2015council
2015council.csv
2016_clfpred
2016_council.csv
2016council
2016council.csv
20_More_EDA.ipynb
21_regress_on_clf_preds-Copy1.ipynb
22_xgboost.ipynb
Houston Yearly Data - Sheet1 (1).

In [4]:
houston_data = pd.read_csv("city_council_data.csv")
houston_data_columns = houston_data.iloc[0,1:].values
houston_data_index = houston_data.iloc[2:,0].values
houston_data  = houston_data.iloc[1:,1:]
houston_data

# houston_data.index= houston_data_index
houston_data.columns = houston_data_columns
# #houston_data.columns = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
# #       '2018', '2019']
# houston_data.fillna(0, inplace=True)
houston_data_index = [str(int(x)) for x in houston_data.index]
houston_data = houston_data.T
houston_data.columns = ['2010', '2011', '2012', '2013', '2014', '2015', '2016']
# #       '2018', '2019']
# houston_data = houston_data.T
houston_data = houston_data.T
houston_data

,Mayor,Controller,City Council District A,City Council District B,City Council District C,City Council District D,City Council District E,City Council District F,City Council District G,City Council District H,City Council District I,City Council District J,City Council District K,City Council At Large 1,City Council At Large 2,City Council At Large 3,City Council At Large 4,City Council At Large 5
2010,Parker,R Green,B Stardig,J Johnson,A Clutterbock,W Adams,M Sullivan,A Hoang,O Pennington,E Gonzalez,J Rodriguez,na,na,S Costello,S Lovell,M Noriega,C Bradford,J Jones
2011,Parker,R Green,B Stardig,J Johnson,A Clutterbock,W Adams,M Sullivan,A Hoang,O Pennington,E Gonzalez,J Rodriguez,na,na,S Costello,S Lovell,M Noriega,C Bradford,J Jones
2012,Parker,R Green,H Brown,J Davis,E Cohen,W Adams,M Sullivan,A Hoang,O Pennington,E Gonzalez,J Rodriguez,M Laster,L Green,S Costello,A Burks,M Noriega,C Bradford,J Christie
2013,Parker,R Green,H Brown,J Davis,E Cohen,W Adams,M Sullivan,A Hoang,O Pennington,E Gonzalez,J Rodriguez,M Laster,L Green,S Costello,A Burks,M Noriega,C Bradford,J Christie
2014,Parker,R Green,B Stardig,J Davis,E Cohen,D Boykins,D Martin,R Nguyen,O Pennington,E Gonzalez,R Gallegos,M Laster,L Green,S Costello,D Robison,M Kubosh,C Bradford,J Christie
2015,Parker,C Brown,B Stardig,J Davis,E Cohen,D Boykins,D Martin,R Nguyen,O Pennington,E Gonzalez,R Gallegos,M Laster,L Green,S Costello,D Robison,M Kubosh,C Bradford,J Christie
2016,Turner,C Brown,B Stardig,J Davis,E Cohen,D Boykins,D Martin,S Le,G Travis,K Cisneros,R Gallegos,M Laster,L Green,M Knox,D Robison,M Kubosh,K Edwards,J Christie


In [ ]:
def add_houston_data(year_df, houston_data, index):
    for col in houston_data.columns:
        if index ==0:
            year_df[col] = 'na'
        else:
            year_df[col] = houston_data.iloc[index][col]
        
    return year_df

In [ ]:
houston_columns = list(houston_data.columns)

In [ ]:
for year_index, year in enumerate(fy_list):
    #fy_list[year_index].drop(['col'], axis=1, inplace=True)
    fy_list[year_index] = add_houston_data(year, houston_data, year_index)
    

In [ ]:
fy_list[1].head()

In [ ]:
houston_columns

In [ ]:
df = pd.concat([year for year in fy_list])
dummies = pd.get_dummies(df[list(houston_data.columns)]) # One-hot encoding of categorical variables
df = pd.concat([dummies, df.drop(list(houston_data.columns), axis=1)], axis=1)
    #fy_list[year_index].drop([list(houston_data.columns)], axis=1, inplace=True)


In [ ]:
for i, y in enumerate(fy_list):
    year = int('201'+str(i))
    fy_list[i] = df.loc[df['Fiscal Year']==year]

In [ ]:
for i, y in enumerate(fy_list):
    fy_list[i].to_csv('201'+str(i)+'_council.csv')

In [ ]:
df_c = pd.read_csv("2012_council.csv")
df_c.head()

### Information Value

In [ ]:
train_iv_list = []
for i, y in enumerate(fy_list):
    train_iv, train_IV_series = data_vars(fy_list[i].drop(['OA_Difference','Actuals','Over Budget','Current Budget'], axis=1),  fy_list[i]['Over Budget'])
    train_iv_list.append([train_iv, train_IV_series])



In [ ]:
train_iv_list[2][1].sort_values(ascending=False, by='IV').loc[train_iv_list[2][1]['STRENGTH']!='useless']



In [ ]:
keep_vars = []
for i, item in enumerate(train_iv_list):
    quantile = train_iv_list[i][1]['IV'].quantile(.6)
    print(quantile)

In [ ]:
train_iv_list[i][1].sort_values(ascending=False, by='IV').loc[train_iv_list[i][1]['IV']<=.002][['VAR_NAME','IV']].head()

In [ ]:
keep_vars = []
for i, item in enumerate(train_iv_list):
    year_keep = list(train_iv_list[i][1].sort_values(ascending=False, by='IV').loc[train_iv_list[i][1]['IV']>=.003]['VAR_NAME'])
    if i==0:
        keep_vars = year_keep
        #print(len(year_keep))
    else:
        keep_vars = list(set(year_keep) | set(keep_vars))
        #print(len(year_keep))
        

drop_vars = set(set(list(fy_list[0].columns)).symmetric_difference(keep_vars))
drop_vars

In [ ]:
pd.DataFrame(pd.Series(keep_vars))

In [ ]:
keep_vars += ['Fiscal Year']

In [ ]:
len(keep_vars)

In [2]:
woe_pca_df_list = []
woe_pca_explained_var_ratio_list = []

for i, y in enumerate(fy_list):
    woe = WoeEncoder(columns=keep_vars, suffix='')
    target = 'Over Budget'
    y = fy_list[i][target]
    
    hold_df = woe.fit_transform(fy_list[i], y)
    
    pca = PCA(n_components=50)
    
    X = hold_df[keep_columns].fillna(0)
    del hold_df; gc.collect()
    
    woe_pca_df_list.append(pca.fit(X))
    woe_pca_explained_var_ratio_list.append(pca.explained_variance_ratio_.sum())
    

NameError: name 'fy_list' is not defined

In [ ]:
%debug


In [ ]:
#12:37 am

In [ ]:
fy_list[1].head()

In [ ]:
# #bar = progressbar.ProgressBar()

# hold_list = []
# for split in splits_to_try:
    
#     hold_dict = {}
    
#     X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
#     y_train = X_train['Over Budget']
#     X_train.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)
#     X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
#     y_test = X_test['Over Budget']
#     y_original_budget = X_test['Original Budget']
#     X_test.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)

#     model = xgb.XGBClassifier() #subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
#     model.fit(X_train,y_train)

#     xgb_test_predictions = model.predict(X_test)
#     xgb_train_predictions = model.predict(X_train)

#     model_test_auc = metrics.roc_auc_score(y_test, xgb_test_predictions)
#     model_train_auc = metrics.roc_auc_score(y_train, xgb_train_predictions)
#     original_budget_auc = metrics.roc_auc_score(y_test, y_original_budget)
    
#     hold_dict.update({'split': split[2]})
#     hold_dict.update({'model_test_auc': model_test_auc})
#     hold_dict.update({'model_train_auc': model_train_auc})
#     hold_dict.update({'original_budget_auc': original_budget_auc})
    
    
#     hold_list.append(hold_dict)
    



In [ ]:
# results_df = pd.DataFrame(hold_list)
# results_df['Model Improvement'] = results_df['model_test_auc']-results_df['original_budget_auc']
# results_df


In [ ]:
# results_df['model_test_auc'].mean()

##adding binary predictor as a feature for XGBReggresor to use later

In [ ]:
hold_list = []
fy_augmented = []
for index, split in enumerate(splits_to_try):
    
    hold_dict = {}
    
    X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
    y_train = X_train['Over Budget']
    X_train.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)
    X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
    y_test = X_test['Over Budget']
    y_original_budget = X_test['Original Budget']
    X_test.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)

    model = xgb.XGBClassifier() #subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
    model.fit(X_train,y_train)

    xgb_test_predictions = model.predict(X_test)
    xgb_train_predictions = model.predict(X_train)
    
    xgb_test_predictions_probability = [x[1] for x in model.predict_proba(X_test)]
    xgb_train_predictions_probability = [x[1] for x in model.predict_proba(X_train)]

    model_test_auc = metrics.roc_auc_score(y_test, xgb_test_predictions)
    model_train_auc = metrics.roc_auc_score(y_train, xgb_train_predictions)
    original_budget_auc = metrics.roc_auc_score(y_test, y_original_budget)
    
    if index <= 5:
        hold_df = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]]) #grab the test dataframe
        hold_df['Over_Budget_Prediction'] = xgb_test_predictions_probability #add in the over under budget predictions
        fy_augmented.append(hold_df) #store it back in the fy_list
    
    
    hold_dict.update({'split': split[2]})
    hold_dict.update({'model_test_auc': model_test_auc})
    hold_dict.update({'model_train_auc': model_train_auc})
    hold_dict.update({'original_budget_auc': original_budget_auc})
    hold_dict.update({'model':model})
    
    
    
    hold_list.append(hold_dict)
    

In [ ]:
results_df = pd.DataFrame(hold_list)
results_df['Model Improvement'] = results_df['model_test_auc']-results_df['original_budget_auc']
results_df


In [ ]:
for i, y in enumerate(fy_list):
    if i ==0:
        None
    else:
        fy_list[i] = fy_augmented[i-1]

In [ ]:
xgb_test_predictions[:10]

In [ ]:
fy_list[6][['Over_Budget_Prediction','OA_Difference']].head(20)

In [ ]:
splits_to_try = [
    
                #predicting off "two years"
                 #[(0,1),(1,2),'10-11 to predict 11'],
                 #[(0,2),(2,3),'10-11 to predict 12'],
                 #[(0,3),(3,4),'10-12 to predict 13'],
                 #[(0,3),(3,5),'10-12 to predict 13 and 14'],
                 [(1,4),(4,5),'11-13 to predict 14'], 
                 #[(1,4),(4,6),'11-13 to predict 14 and 15'],
                 [(2,5),(5,6),'12-14 to predict 15'],
                 #[(2,5),(5,7),'12-14 to predict 15 and 16'],
                 [(3,6),(6,7),'13-15 to predict 16'], 
    
                #predicting off "three" years
    
                 #[(0,4),(5,6),'10-13 to predict 14'],
                 #[(0,4),(4,6),'10-13 to predict 14 and 15'],
                 #[(0,4),(4,7),'10-13 to predict 14 and 15 and 16'],
                 [(1,5),(5,6),'11-14 to predict 15'],
                 #[(1,5),(5,7),'11-14 to predict 15 and 16'],
                 [(2,6),(6,7),'12-15 to predict 16']
                ]
        

#predicting Actuals with binary classifier "over budget" prediction as a predictor

In [ ]:
hold_list = []
for split in splits_to_try:
    
    hold_dict = {}
    
    X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
    y_train = X_train['Actuals']
    X_train.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1, inplace=True)
    X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
    y_test = X_test['Actuals']
    y_original_budget = X_test['Original Budget']
    X_test.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1, inplace=True)

    model = xgb.XGBRegressor(subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
    model.fit(X_train,y_train)

    xgb_test_predictions = model.predict(X_test)
    xgb_train_predictions = model.predict(X_train)

    model_test_RMSE = np.sqrt(np.mean((xgb_test_predictions-y_test)**2))
    model_train_RMSE = np.sqrt(np.mean((xgb_train_predictions-y_train)**2))
    original_budget_RMSE = np.sqrt(np.mean((y_original_budget-y_test)**2))

    hold_dict.update({'split': split[2]})
    hold_dict.update({'model_test_RMSE': model_test_RMSE})
    hold_dict.update({'model_train_RMSE': model_train_RMSE})
    hold_dict.update({'original_budget_RMSE': original_budget_RMSE})
    
    
    hold_list.append(hold_dict)
    

In [ ]:
results_df = pd.DataFrame(hold_list)
results_df['Model Improvement'] = results_df['original_budget_RMSE']-results_df['model_test_RMSE']
results_df


# stop here

In [1]:
fy_list[0].head()

NameError: name 'fy_list' is not defined

In [ ]:
splits_to_try_2 = [
    
                #predicting off "two years"
    
                 [(0,3),(3,4),'10-12 to predict 13'],
                 [(0,3),(3,5),'10-12 to predict 13 and 14'],
                 [(1,4),(4,5),'11-13 to predict 14'], 
                 [(1,4),(4,6),'11-13 to predict 14 and 15'],
                 [(2,5),(5,6),'12-14 to predict 15'],
                 [(2,5),(5,7),'12-14 to predict 15 and 16'],
                 [(3,6),(6,7),'13-15 to predict 16'], 
    
                #predicting off "three" years
    
                 [(0,4),(5,6),'10-13 to predict 14'],
                 [(0,4),(4,6),'10-13 to predict 14 and 15'],
                 [(0,4),(4,7),'10-13 to predict 14 and 15 and 16'],
                 [(1,5),(5,6),'11-14 to predict 15'],
                 [(1,5),(5,7),'11-14 to predict 15 and 16'],
                 [(2,6),(6,7),'12-15 to predict 16']
    
                ]

In [ ]:


hold_list = []
for split in splits_to_try_2:
    
    hold_dict = {}
    
    X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
    y_train = X_train['Over Budget']
    X_train.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)
    X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
    y_test = X_test['Over Budget']
    y_original_budget = X_test['Original Budget']
    X_test.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)

    model = xgb.XGBClassifier() #subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
    model.fit(X_train,y_train)

    xgb_test_predictions = model.predict(X_test)
    xgb_train_predictions = model.predict(X_train)

    model_test_auc = metrics.roc_auc_score(y_test, xgb_test_predictions)
    model_train_auc = metrics.roc_auc_score(y_train, xgb_train_predictions)
    original_budget_auc = metrics.roc_auc_score(y_test, y_original_budget)
    
    hold_dict.update({'split': split[2]})
    hold_dict.update({'model_test_auc': model_test_auc})
    hold_dict.update({'model_train_auc': model_train_auc})
    hold_dict.update({'original_budget_auc': original_budget_auc})
    
    
    hold_list.append(hold_dict)
    

In [ ]:
results_df = pd.DataFrame(hold_list)
results_df['Model Improvement'] = results_df['model_test_auc']-results_df['original_budget_auc']
results_df[['model_test_auc','model_train_auc','split']]


In [ ]:
results_df['model_test_auc'].mean()


In [ ]:


hold_list = []
for split in splits_to_try[1:4]
    
hold_dict = {}
    
    X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
    y_train = X_train['Over Budget']
    X_train.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)
    X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
    y_test = X_test['Over Budget']
    y_original_budget = X_test['Original Budget']
    X_test.drop(['OA_Difference','Actuals','Over Budget','Current Budget'] + drop_list, axis=1, inplace=True)

    model = xgb.XGBClassifier() #subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
    model.fit(X_train,y_train)

    xgb_test_predictions = model.predict(X_test)
    xgb_train_predictions = model.predict(X_train)

    model_test_auc = metrics.roc_auc_score(y_test, xgb_test_predictions)
    model_train_auc = metrics.roc_auc_score(y_train, xgb_train_predictions)
    original_budget_auc = metrics.roc_auc_score(y_test, y_original_budget)
    
    hold_dict.update({'split': split[2]})
    hold_dict.update({'model_test_auc': model_test_auc})
    hold_dict.update({'model_train_auc': model_train_auc})
    hold_dict.update({'original_budget_auc': original_budget_auc})
    
    
    hold_list.append(hold_dict)
    
    


In [ ]:
results_df = pd.DataFrame(hold_list)
results_df['Model Improvement'] = results_df['model_test_auc']-results_df['original_budget_auc']
results_df


In [ ]:
results_df['Model Improvement'].sum()

In [ ]:
hold_list = []
for split in splits_to_try:
    
    hold_dict = {}
    
    X_train = pd.concat([x for x in fy_list[split[0][0]:split[0][1]]])
    y_train = X_train['Actuals']
    X_train.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1, inplace=True)
    X_test = pd.concat([x for x in fy_list[split[1][0]:split[1][1]]])
    y_test = X_test['Actuals']
    y_original_budget = X_test['Original Budget']
    X_test.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1, inplace=True)

    model = xgb.XGBClassifier(subsample= 0.8, min_child_weight= 1, max_depth= 4, gamma= 1, colsample_bytree= 1.0)
    model.fit(X_train,y_train)

    xgb_test_predictions = model.predict(X_test)
    xgb_train_predictions = model.predict(X_train)

    model_test_auc = metrics.roc(y_test, xgb_test_predictions)
    model_train_auc = metrics.roc(y_train, xgb_train_predictions)
    original_budget_auc = metrics.roc(y_test, y_original_budget)
    
    hold_dict.update({'split': split[2]})
    hold_dict.update({'model_test_auc': model_test_auc})
    hold_dict.update({'model_train_auc': model_train_auc})
    hold_dict.update({'original_budget_auc': original_budget_auc})
    
    
    hold_list.append(hold_dict)
    
   




In [ ]:
results_df = pd.DataFrame(hold_list)
results_df['Model Improvement'] = results_df['model_test_auc']-results_df['original_budget_auc']
results_df


In [ ]:
results_df.loc[[0,2,4,6,7,10]]['Model Improvement']


In [ ]:
results_df = pd.DataFrame(hold_list)
results_df

In [ ]:
hold_dict.keys()

In [ ]:
len(res)

In [ ]:
model_sum_error = np.sum(abs(xgb_test_predictions-y_test))
model_sum_error

In [ ]:
budget_sum_error = np.sum(abs(y_original_budget-y_test))
budget_sum_error

### Model precision over budget

In [ ]:
print('$',round(budget_sum_error-model_sum_error,2))

In [ ]:
y_test.std()

In [ ]:
sns.set_style('whitegrid')

x = np.linspace(-1000000000,1000000000,1000000000)
plt.figure(figsize=(12,12))
plt.scatter(y_train, xgb_train_predictions, c='red', alpha=.95)
plt.scatter(y_test, xgb_test_predictions, c='blue', alpha=.95)
plt.scatter(y_test, y_original_budget, c='green', alpha=.95)
plt.plot(x,x,c='black')
plt.xlim([-5,1000000000])
plt.ylim([-5,1000000000])
plt.xlabel('observations')
plt.ylabel('predictions')


In [ ]:
weight_dict = model.get_booster().get_score(importance_type="weight")
weight_df = pd.DataFrame(weight_dict, index=range(len(weight_dict)))
weight_df.T[0].sort_values(ascending=False)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test , xgb_test_predictions)

In [ ]:

sns.set_style('darkgrid')
sns.set(font_scale=2.5)
bins = np.arange(0,90000,90000/50)
 
fig, ax = plt.subplots(figsize=(20,15))

sns.distplot(y_test, label='2015 Actuals',color='green', norm_hist=True, bins=bins, kde=False,  ax = ax)
sns.distplot(xgb_test_predictions, label='XGB_test_predictions 2015',color='black', norm_hist=True, bins=bins, kde=True,  ax = ax)
sns.distplot(y_original_budget, label='2015_original_budget',color='blue',norm_hist=True, bins=bins, kde=True,  ax = ax)
    
ax.set_title(" ")
ax.set_ylabel(" ")
ax.set_xlabel(" ")
ax.set_xlim(0,90000)
ax.legend()

In [ ]:
original_budget_residuals = fy_15['Original Budget']-fy_15['Actuals']
model_residuals = xgb_test_predictions-y_test
hold_df = pd.DataFrame()
hold_df['Original Budget Residuals'] = original_budget_residuals
hold_df['Model Residuals'] = model_residuals


In [ ]:
np.sqrt(np.mean(model_residuals**2))

In [ ]:
np.sqrt(np.mean(original_budget_residuals**2))

In [ ]:
model_residuals_series = pd.Series(model_residuals)

In [ ]:
original_budget_residuals_series =pd.Series(original_budget_residuals)

In [ ]:
residuals_df = pd.DataFrame()
residuals_df['Model_Residuals'] = model_residuals_series
residuals_df['Original_Budget_Residuals'] = original_budget_residuals_series

extreme_residuals = residuals_df.loc[(residuals_df['Model_Residuals']>=80000) | (residuals_df['Model_Residuals']<=-80000) | (residuals_df['Original_Budget_Residuals']>=80000) | (residuals_df['Original_Budget_Residuals']<=-80000) ] 
residuals_under_80K = residuals_df.loc[(residuals_df['Model_Residuals']<80000) & (residuals_df['Model_Residuals']>-80000) & (residuals_df['Original_Budget_Residuals']<=80000) & (residuals_df['Original_Budget_Residuals']>=-80000) ] 


In [ ]:
print('Model Extreme Resisduals:           ',int(np.sqrt(np.mean(extreme_residuals['Model_Residuals']**2))), '\nOriginal Budget Extreme Resisduals: ',  int(np.sqrt(np.mean(extreme_residuals['Original_Budget_Residuals']**2))))
print(' ')
print('Model Under 80K Resisduals:           ',int(np.sqrt(np.mean(residuals_under_80K['Model_Residuals']**2))), '\nOriginal Budget Under 80K Resisduals: ',  int(np.sqrt(np.mean(residuals_under_80K['Original_Budget_Residuals']**2))))


In [ ]:
sns.set_style('darkgrid')
sns.set(font_scale=2.5)
 
fig, ax = plt.subplots(figsize=(20,15))
bins_ = np.arange(-80000,80000,80000/50)

sns.distplot(model_residuals, label='Model Residuals', norm_hist=True, bins = bins_, color='green',   ax = ax)
sns.distplot(original_budget_residuals, label='Original Budget Residuals',bins=bins_, color='black',  ax = ax)
    
ax.set_title(" ")
ax.set_ylabel(" ")
ax.set_xlabel(" ")
ax.set_xlim(-80000,80000)
ax.legend()

In [ ]:
#reference: https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


train_df = pd.concat([fy_list[0], fy_list[1], fy_list[2], fy_list[3], fy_list[4]])
Y = train_df['Actuals'].values
X = train_df.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1)
test_df = pd.concat([fy_list[5], fy_list[6]])
test = test_df.drop(['OA_Difference','Actuals','Current Budget'] + drop_list, axis=1)
y_test = test_df['Actuals']
original_budget = test_df['Original Budget']

In [ ]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [ ]:
model = xgb.XGBRegressor(learning_rate=.02, n_estimators=600, silent=True, nthread=1)

#xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
  #                  silent=True, nthread=1)

In [ ]:
folds = 3
param_comb = 5

#skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, n_jobs=4, verbose=3, random_state=1001 )

#cv=skf.split(X,Y)
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, Y)
timer(start_time) # timing ends here for "start_time" variable

In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

In [ ]:
xgb_test_predictions = random_search.predict(test)
#results_df = pd.DataFrame(data={'Actuals':y_test[:,1]})
#results_df.to_csv('submission-random-grid-search-xgb-porto-01.csv', index=False)


In [ ]:
model_test_RMSE = np.sqrt(np.mean((xgb_test_predictions-y_test)**2))
#model_train_RMSE = np.sqrt(np.mean((xgb_train_predictions-y_train)**2))
original_budget_RMSE = np.sqrt(np.mean((original_budget-y_test)**2))


In [ ]:
y_test.shape


In [ ]:
len(y_original_budget)


In [ ]:
a =pd.Series([7,8,9,10])
b=pd.Series([1,2,1])
np.mean([36,36,64])
np.mean((a-b)**2)
original_budget[:10]


In [ ]:
y_test[:10]

In [ ]:
pd.DataFramey_original_budget[:1000].values



In [ ]:
y_test.std()


In [ ]:
model_test_RMSE


In [ ]:
original_budget_RMSE


In [ ]:
weight_dict = random_search.best_estimator_.get_booster().get_score(importance_type="weight")
weight_df = pd.DataFrame(weight_dict, index=range(len(weight_dict)))
weight_df.T[0].sort_values(ascending=False)


In [ ]:
len(y_original_budget)

In [ ]:
len(xgb_test_predictions)

In [ ]:
len(y_test)

In [ ]:
sns.set_style('whitegrid')

x = np.linspace(-1000000000,1000000000,1000000000)
plt.figure(figsize=(12,12))
#plt.scatter(y_train, xgb_train_predictions, c='red', alpha=.95)
plt.scatter(y_test, xgb_test_predictions, c='blue', alpha=.95)
plt.scatter(y_test, y_original_budget, c='green', alpha=.95)
plt.plot(x,x,c='black')
plt.xlim([-5,1000000000])
plt.ylim([-5,1000000000])
plt.xlabel('observations')
plt.ylabel('predictions')


# H2o

In [ ]:
h2o.init()

In [ ]:
# create h2o frame for training data
training_frame = pd.concat([X_train,y_train], axis=1)
training_frame.columns = list(training_frame.columns[:-1]) + ['Actuals']
training_frame = h2o.H2OFrame(training_frame)

In [ ]:
# create h2o frame for testing data
testing_frame = pd.concat([X_test,y_test], axis=1)
testing_frame.columns = list(testing_frame.columns[:-1]) + ['Actuals']
testing_frame = h2o.H2OFrame(testing_frame)

In [ ]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(None, 'Actuals', training_frame)

In [ ]:
y_test.std()

In [ ]:
predicted_actuals = aml.predict(h2o.H2OFrame(pd.concat([X_test, y_test], axis=1)))
np.sqrt(np.mean((predicted_actuals['predict'].as_data_frame()['predict'].values-y_test)**2))

In [ ]:
aml.leader

In [ ]:
#3:55 pm

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_train, aml.predict(training_frame).as_data_frame()['predict'].values, c='red')
plt.scatter(y_test, aml.predict(testing_frame).as_data_frame()['predict'].values, c='blue')
#plt.scatter(train_df['voyage_duration'], train_df['eta_diff'], c='green')
plt.plot(x,x,c='black')
plt.xlim([-5,100000000])
plt.ylim([-5,100000000])
plt.xlabel('observations')
plt.ylabel('predictions')

In [ ]:
#9/4 4:42pm